<a href="https://colab.research.google.com/github/psuarezserrato/aprendizaje-geometrico/blob/main/notebooks/AGP-FAUST-EulerChar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

path_to_zip_file = '/content/drive/MyDrive/MPI_FAUST/MPI-FAUST.zip'

directory_to_extract_to = '/content'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
  zip_ref.extractall(directory_to_extract_to)


In [ ]:
!apt install libspatialindex-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 kB]
Fetched 555 kB in 1s (854 kB/s)
Selecting previously unselected package libspatialindex4v5:amd64.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../libspatialindex4v5_1.8.5-5_

In [ ]:
!pip install Rtree trimesh

     |████████████████████████████████| 1.0MB 37.6MB/s 
     |████████████████████████████████| 634kB 39.4MB/s 


In [ ]:
import os
import trimesh
import time
import json
import numpy as np

trimesh.proximity.max_tangent_sphere

Find the center and radius of the sphere which is tangent to the mesh at the given point and at least one more point with no non-tangential intersections with the mesh.

Masatomo Inui, Nobuyuki Umezu & Ryohei Shimane (2016) Shrinking sphere: A parallel algorithm for computing the thickness of 3D objects, Computer-Aided Design and Applications, 13:2, 199-207, DOI: 10.1080/16864360.2015.1084186

Parameters
points ((n, 3) float) – Points in space.

inwards (bool) – Whether to have the sphere inside or outside the mesh.

normals ((n, 3) float or None) – Normals of the mesh at the given points if is None computed automatically.

Returns
centers ((n,3) float) – Centers of spheres

radii ((n,) float) – Radii of spheres

In [ ]:
# Obtenemos los datos de base de datos MPI-FAUST
files = os.listdir('/content/MPI-FAUST/training/registrations/')

# Calculate the Euler characteristic
euler_characterisc = dict()
reaches = dict()
for file_name in files:
  print(file_name)
  if ".ply" in file_name:
    # Leemos los datos
    mesh = trimesh.load_mesh(f'/content/MPI-FAUST/training/registrations/{file_name}')
    # Euler characteric
    euler_characterisc.update({file_name: mesh.euler_number})
    # Calculamos las esferas tangentes.
    mv = trimesh.proximity.max_tangent_sphere(mesh, mesh.vertices)
    reach = np.min(mv[1])
    # Calculamos el alcance
    reaches.update({file_name: reach})
    # Engordamos the mesh
    new_mesh_vertices = mesh.vertices+reach*mesh.vertex_normals
    new_mesh = trimesh.Trimesh(vertices=new_mesh_vertices, faces=mesh.faces, process=False)
    new_mesh.export(file_obj=f'/content/fatten/{file_name}')
  else:
    pass


tr_reg_053.png
tr_reg_055.ply


FileNotFoundError: ignored

In [ ]:
with open('/content/reaches.json', 'w') as fp:
    json.dump(reaches, fp)

In [ ]:
with open('/content/euler_characterisc.json', 'w') as fp:
    json.dump(euler_characterisc, fp)

In [ ]:
!cp -r "/content/fatten" "/content/drive/MyDrive/MPI_FAUST/fatten_reaches"

In [ ]:
euler_characterisc

In [ ]:
path = "/content/MPI-FAUST/training/registrations/tr_reg_067.ply"

mesh = trimesh.load_mesh(path)
mesh.show()